## Keras Layered Model & Explorations - JoanieW

Based on: https://elix-tech.github.io/ja/2016/06/02/kaggle-facial-keypoints-ja.html#conv

And modeled after Jackie's code structure for Lenet5

In [17]:
import os
import imp
import pickle
from utils import predict_models, load_models, transform_data


In [13]:
cd ..

/Users/joanieweaver/Desktop/intro_ml/blackboxes


In [24]:
### this is gold right here.
imp.reload(load_models)
file_path = "cleantrain/"
trainer = load_models.LoadTrainModels("data/models/", file_path)

trainer.print_paths()

Model dir: data/models/
Pickle dir: cleantrain/


### For every version of a clean Train file in a given path, create and save a model

In [25]:

imp.reload(load_models)

file_path = "cleantrain/"
#files = os.listdir("../CleanTrain")
files = os.listdir(file_path)

#For every version of a cleaned Train file in CleanTrain directory, create and save a model
for filename in files: 
    print("Opening file: ", filename)
    clean_file = "".join((file_path,filename))
    train_data = pickle.load( open( clean_file, "rb" ) )
    train_data = train_data.drop(['level_0', 'check_sum', 'index'], axis=1,errors='ignore')
    print("Train Shape:", train_data.shape)

    filename = str(filename).replace('.p', '').strip()
    print("Begin model and train:")
    model_name = "".join((filename,"_jcw"))
    print("Model name:", model_name)
    model, history = trainer.train_model(model_name, train_data,verbose = True)
    print("End model and train")    
    print()

Opening file:  clean_w_dups.p
Train Shape: (6488, 31)
Begin model and train:
Model name: clean_w_dups_jcw
Scaling 6488 images...
Scaling of 6488 observations complete.
Begining the split of Train with all features
Looking for model JW
Loading model: clean_w_dups_jcw
End model and train

Opening file:  clean_w_outliers.p
Train Shape: (7041, 31)
Begin model and train:
Model name: clean_w_outliers_jcw
Scaling 7041 images...
Scaling of 7041 observations complete.
Begining the split of Train with all features
Looking for model JW
Loading model: clean_w_outliers_jcw
End model and train

Opening file:  clean_o_dups.p
Train Shape: (6483, 31)
Begin model and train:
Model name: clean_o_dups_jcw
Scaling 6483 images...
Scaling of 6483 observations complete.
Begining the split of Train with all features
Looking for model JW
Loading model: clean_o_dups_jcw
End model and train

Opening file:  clean_wo_dups.p
Train Shape: (6478, 31)
Begin model and train:
Model name: clean_wo_dups_jcw
Scaling 6478 ima

### For every model file in a given path, predict using the model and save the predictions in CSV file

In [26]:
imp.reload(predict_models)
imp.reload(transform_data)

id_lookup = pickle.load( open( "data/id_lookup.p", "rb" ) )
test = pickle.load( open( "data/test.p", "rb" ) )

#Using local paths as this is way faster...
file_path = "data/models/"
pred_path = "data/predictions/"

predictor = predict_models.PredictModels(file_path,pred_path , id_lookup)

predictor.print_paths()

files = os.listdir(file_path)
#For every model in file_path, predict using the model and save the predictions in CSV file
for filename in files:
    if ".h5" in filename:
        base_name = filename[:-3]
        model_json = ''.join((base_name,".json"))
        print("Working with: ", base_name)
        print("Begin Predict")
        Y= predictor.predict_model(base_name, filename, model_json, test)
        print("End model and train")    
        print()


Model dir: data/models/
Pickle dir: data/predictions/
Working with:  clean_test_jcw
Begin Predict


AttributeError: 'PredictModels' object has no attribute 'predict_model'

In [ ]:
#From Sandip's models
def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance = metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error = metrics.mean_absolute_error(y_true, y_pred) 
    mse = metrics.mean_squared_error(y_true, y_pred) 
    #mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error = metrics.median_absolute_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
    #print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))